1. Install Library

In [1]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 13.6 MB/s 


In [2]:
pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 830 kB 16.1 MB/s 
     |████████████████████████████████| 280 kB 38.4 MB/s 
     |████████████████████████████████| 1.6 MB 54.7 MB/s 
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=44fa05be5a1e0bd46f369f877ce6b80bce8347ef6965d5f725ade38bc727fd16
  Stored in directory: /root/.cache/pip/wheels/08/66/b4/921e351e63d88696932279d6163e125727c9da70ed8ca38419
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


2. Import Library And Download

In [3]:
import pandas as pd 
import numpy as np
import nltk
import string 
import re
import swifter
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

3. Load Data And Put On Data Frame

In [ ]:
readJson = pd.read_json("banjirJakarta2022sept-nov.json",orient='records', lines=True)

TWEET_DATA = pd.DataFrame(readJson)
TWEET_DATA.head(5)

4. Case Folding

In [ ]:
TWEET_DATA['tweet'] = TWEET_DATA['text'].str.lower()
TWEET_DATA.head(5)

5. Remove Special Character Include URL, Mention, Hashtag, Space, Punctuation, Single Char

In [ ]:
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#_][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_singl_char)

TWEET_DATA.head(5)

6.Tokenizing

In [ ]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(word_tokenize_wrapper)

TWEET_DATA.head(5)

7. Remove Stopwords

In [ ]:
# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# # ----------------------- add stopword from txt file ------------------------------------
# # read txt stopword using pandas
# txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# # convert stopword string to list & append additional stopword
# list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# # ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(stopwords_removal)
TWEET_DATA.head(5)

8. Normalization

In [ ]:
normalizad_word = pd.read_excel("normalisasi.xlsx")
# TWEET_DATA.drop(columns=['tweet_normalized','tweet_tokens_stemmed'], axis=1,inplace=True)

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(normalized_term)

TWEET_DATA.head(5)

9. Stemming

In [ ]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in TWEET_DATA['tweet']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].swifter.apply(get_stemmed_term)
TWEET_DATA.head(5)

10. Labelling

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import reprlib
nltk.download('vader_lexicon')

# Memanfaatkan nltk VADER untuk menggunakan leksikon kustom
sia1A, sia1B = SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer()
# membersihkan leksikon VADER default
sia1A.lexicon.clear()
sia1B.lexicon.clear()

# Membaca leksikon InSet
# Leksikon InSet lexicon dibagi menjadi dua, yakni polaritas negatif dan polaritas positif;
# kita akan menggunakan nilai compound saja untuk memberi label pada suatu kalimat
with open('dictset-neg.json') as f:
    data1A = f.read()
with open('dictset-pos.json') as f:
    data1B = f.read()

# Mengubah leksikon sebagai dictionary
insetNeg = json.loads(data1A)
insetPos = json.loads(data1B)

# Update leksikon VADER yang sudah 'dimodifikasi'
sia1A.lexicon.update(insetNeg)
sia1B.lexicon.update(insetPos)

In [ ]:
def is_positive_inset(tweet: str) -> int:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia1A.polarity_scores(tweet)["compound"] + sia1B.polarity_scores(tweet)["compound"] > 0

In [ ]:
TWEET_DATA["textjoin"] = TWEET_DATA["tweet"].apply(lambda x : " ".join(x))
tweets = TWEET_DATA["textjoin"]

for ind in range(len(TWEET_DATA)):
  if is_positive_inset(TWEET_DATA.loc[ind,"textjoin"]) == True:
    TWEET_DATA.loc[ind,"sentimen"] = 'Positif'
  else:
    TWEET_DATA.loc[ind,"sentimen"] = 'Negatif'


TWEET_DATA